## **InVEST NDR Iterative Model Execution**

publication: "Addressing spatial and temporal data scarcity in water quality monitoring using machine learning for long-term ecosystem service calibration and validation"

author: "Valladares-Castellanos M, De Jesus Crespo R, Douthat T"


github: https://github.com/LSU-EPG/Puerto-Rico-ES-Project/tree/main/Data_Scarcity_Framework

**Summary**: This notebook outlines the procedure for performing batch calibration of the InVEST (Integrated Valuation of Ecosystem Services and Tradeoffs) Nutrient Delivery Ratio model (Section XX). The core of the notebook applies a looped calibration function to optimize the threshold flow accumulation and Borselli K parameters for different land use/land cover (LULC) maps between 1951 and 2000. Calibration is conducted separately for both types of reference watershed clusters.

#### 1. Install the packages

Refer to the following Natural Capital Project link for information about installing natcap.invest packages, dependencies, and batch prcessing:
https://invest.readthedocs.io/en/latest/scripting.html

In [ ]:
import logging
import sys
import natcap.invest.ndr.ndr
import natcap.invest.utils

#### 2. Iterative InVEST NDR Batch model execution

In [ ]:
## set up the parameters 
LOGGER = logging.getLogger(__name__)
root_logger = logging.getLogger()

handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
    fmt=natcap.invest.utils.LOG_FMT,
    datefmt='%m/%d/%Y %H:%M:%S ')
handler.setFormatter(formatter)
logging.basicConfig(level=logging.INFO, handlers=[handler])

# Set the parameters folder location
args = {
    'biophysical_table_path': 'path/biophysical_table.csv',
    'calc_n': True,  #If interested in estimating nitrogen
    'calc_p': True,  #If interested in estimating phosphorus
    'dem_path': 'path/DEM.tif',
    'subsurface_critical_length_n': 'value',
    'subsurface_eff_n': 'value',
    'workspace_dir':'workspace_directory_path',
}

# Set the values for the TFA, Borselli K calibration parameters and the timeframe of interest based on the LULC maps and reference 
# watershed clusters
years = [1951, 1977, 1991, 2000]
kb_cal = [0.50, 1.0, 1.50,2.0, 2.5, 3.0, 3.5, 4.0]
tfa_cal = [10, 50,100,150,200,250, 300,350,400,450,500,600,700,800,900,1000]
watershed_cluster = [1,2]

# Loops through the values 
for tfa in tfa_cal:
    args['threshold_flow_accumulation'] = tfa
    for kb in kb_cal:
        args['k_param'] = kb
        for year in years:
            lulc = 'path/LULC_' + str(year) + '.tif'
            runoff = 'path/Q_mm_' + str(year) + '_' + '.tif'
            args['lulc_path'] = lulc
            args['runoff_proxy_path'] = runoff
            for cluster in watershed_cluster:
                ref_watershed = 'path/Ref_WA_Cluster' + str(cluster) + '.shp' #reference watersheds shapefile
                args['watersheds_path'] = ref_watershed
                args['results_suffix'] = 'year' + str(year) + '_' + 'cluster' + str(cluster) + '_' + 'TFA' + str(tfa) + '_' + 'kb' + str(kb)
                if __name__ == '__main__':
                    natcap.invest.ndr.ndr.execute(args)

#### 3. References:

Campanhão, L. M. B., & Ranieri, V. E. L. (2023). Influence of forest proportion and configuration at the watershed and riparian zone scales on sediment yield: a simulation experiment. Landscape Ecology, 38(11), 2839-2860. https://doi.org/10.1007/s10980-023-01751-6

Natural Capital Project. (2023). InVEST 3.14.0. In Stanford University, University of Minnesota, Chinese Academy of Sciences, The Nature Conservancy, World Wildlife Fund, Stockholm Resilience Centre and the Royal Swedish Academy of Sciences. https://naturalcapitalproject.stanford.edu/software/invest

Valladares-Castellanos, M., de Jesús Crespo, R., Xu, Y. J., & Douthat, T. H. (2024). A framework for validating watershed ecosystem service models in the United States using long-term water quality data: Applications with the InVEST Nutrient Delivery (NDR) model in Puerto Rico. Science of the Total Environment, 949, 175111. https://doi.org/https://doi.org/10.1016/j.scitotenv.2024.175111 